<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/Mlp_Bethelegend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tqdm
import os

In [2]:
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
print(os.getcwd())

/content


In [3]:
data = pd.read_excel("/content/2021 날짜별 타자 데이터.xlsx")
hitter = pd.read_excel("/content/2021 타자 명단.xlsx")
vs_avg_data = pd.read_excel("/content/팀 상대 타율 정렬.xlsx")
pitcher_stadium = pd.read_excel("/content/2021 타자 로그 데이터.xlsx")

In [4]:
data = pd.DataFrame(data)
hitter = pd.DataFrame(hitter)
vs_avg_data = pd.DataFrame(vs_avg_data)
pitcher_stadium = pd.DataFrame(pitcher_stadium)

In [ ]:
#일단 이 함수는 나중에,,
def make_pitcher_stadium(record_data, pitcher_stadium):
  day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  name = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  vs = np.empty(len(record_data), dtype=np.chararray)
  over
  stadium_avg = np.zeros(len(record_data))
  
  
  vs = np.empty(len(record_data), dtype=np.chararray)
  vs_ab = np.zeros(len(record_data))
  vs_hit = np.zeros(len(record_data))

In [ ]:
print(data.head(), len(data))

In [ ]:
print(hitter.head(), len(hitter))

In [ ]:
data.loc[:,['day','name','ab','hit']].dtypes

In [ ]:
data.loc[:,['day','name','ab','hit']]

In [5]:
# 더블헤더 처리
temp = np.zeros(len(data))

# day, name이 같은 값이 있다는 것은 더블헤더 경기가 있다는것
# keep=False 옵션은 중복 값에 해당하는 튜플들을 모두 True로 리턴, 그 외 옵션 : 'first', 'last' 의 경우 둘 중 하나만 True
dup = data.loc[:,['day','name']].duplicated(keep=False) 

for i in range(len(data)):
  if dup[i]==False:
    temp[i]=0
  elif dup[i]==True:
    temp[i]=1

# 중복의 첫번째 행은 더블헤더 1차전
# 중복인 두번째 행이 더블헤더 2차전이다.
for i in range(1,len(temp)):
  if temp[i]==1 and temp[i-1]==1:
    temp[i]=2

data['double']=temp

In [6]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, day=5):

  # 최근 5일(기본) 경기 타율 저장할 공간 생성
  recent_days_avg = np.zeros(len(record_data)) # 최근 5일(기본) 경기 타율
  recent_days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_days_day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  recent_days_double = np.zeros(len(record_data), dtype=np.int8) # 더블헤더 분류 / 0 : 더블헤더 아님 / 1 : 더블헤더 1차전 / 2 : 더블헤더 2차전

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h].reset_index() # 특정 타자 한명에 대해서만 데이터 추출
    for i in range(len(hdata['day'])): # 추출한 데이터에서 한 행씩 접근
      recent_days_hitter[ps] = h # 현재 타자
      r = hdata.loc[i,'day']  # 현재 기준 날짜
      recent_days_day[ps] = r
      d = hdata.loc[i,'double'] # 현재 더블헤더 분류
      recent_days_double[ps] = d      
      if d==2: # 더블헤더 2차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r].loc[hdata['double']!=2] # r 날짜까지 가져와서 2차전을 제외
      else: # 더블헤더가 아니고, 더블헤더 1차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기
         
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_days_avg[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = recent_days_day
  result_Dataframe['name'] = recent_days_hitter
  result_Dataframe['recent_days_avg'] = recent_days_avg
  result_Dataframe['double'] = recent_days_double

  return result_Dataframe

In [7]:
#상대 타율 구하는 함수
#데이터 파일에서 vs 내림차순 후, 누적 타수 / 누적 안타 계산이 끝난 상태여야함
def make_vs_avg(hitter_data, record_data):
  vs_avg = np.zeros(len(record_data))
  name = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  vs = np.empty(len(record_data), dtype=np.chararray)
  vs_ab = np.zeros(len(record_data))
  vs_hit = np.zeros(len(record_data))
  double_h = np.zeros(len(record_data))

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r, d, v,ab, hit in zip(hdata['day'], hdata['double'], hdata['vs'], hdata['vs_cu_ab'], hdata['vs_cu_hit']): # 추출한 데이터에서 한 행씩 접근
      day[ps] = r  # 현재 기준 날짜
      name[ps] = h # 현재 타자
      vs[ps] = v
      vs_ab[ps] = ab
      vs_hit[ps] = hit
      double_h[ps] = d
      try:
        vs_avg[ps] = hit / ab
      except ZeroDivisionError:
        vs_avg[ps] = 0
      ps+=1
  
  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = day
  result_Dataframe['double'] = double_h
  result_Dataframe['name'] = name
  result_Dataframe['vs_team'] = vs
  result_Dataframe['vs_ab'] = vs_ab
  result_Dataframe['vs_hit'] = vs_hit
  result_Dataframe['vs_avg'] = vs_avg
  result_Dataframe

  return result_Dataframe



In [8]:
vs_avg = make_vs_avg(hitter, vs_avg_data)

100%|██████████| 90/90 [00:00<00:00, 772.64it/s]


In [ ]:
vs_avg.loc[vs_avg['name']=='전준우']

In [9]:
recent_days = make_recent_days(hitter, data)

100%|██████████| 90/90 [00:09<00:00,  9.80it/s]


In [ ]:
recent_days.loc[recent_days['name']=='전준우']

In [10]:
merge_data1 = pd.merge(data, vs_avg, on = ['day', 'name', 'double'], how = 'inner')

In [ ]:
merge_data1.loc[merge_data1['name']=='전준우']

In [11]:
merge_data2 = pd.merge(merge_data1, recent_days, on = ['day', 'name', 'double'], how = 'inner')

In [12]:
len(data), len(recent_days), len(vs_avg), len(merge_data1), len(merge_data2)

(5296, 5296, 5296, 5296, 5296)

In [ ]:
merge_data2

In [13]:
our_data = merge_data2.loc[:, ['day', 'double', 'name', 'vs', 'ab','hit', 'avg', 'slg', 'vs_ab', 'vs_hit', 'vs_avg', 'recent_5','recent_10','success_recent_10','recent_days_avg', 'result']]

In [ ]:
our_data.loc[our_data['name']=='전준우']

In [14]:
mlp_data = our_data.loc[:, ['name', 'vs', 'avg', 'slg', 'vs_ab', 'vs_hit', 'vs_avg', 'recent_5','recent_10','success_recent_10','recent_days_avg', 'result']]

In [ ]:
type(mlp_data), mlp_data.shape

(pandas.core.frame.DataFrame, (5296, 12))

In [15]:
mlp_data

,name,vs,avg,slg,vs_ab,vs_hit,vs_avg,recent_5,recent_10,success_recent_10,recent_days_avg,result
0,강민호,키움,0.000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.000000,0
1,강민호,키움,0.286,0.286,3.0,0.0,0.000000,0.000000,0.000,0.00,0.000000,1
2,강민호,두산,0.250,0.250,0.0,0.0,0.000000,0.285714,0.286,0.50,0.285714,0
3,강민호,두산,0.250,0.333,1.0,0.0,0.000000,0.250000,0.250,0.33,0.250000,1
4,강민호,두산,0.333,0.667,5.0,1.0,0.200000,0.250000,0.250,0.50,0.250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5291,힐리,삼성,0.254,0.386,27.0,6.0,0.222222,0.210526,0.179,0.50,0.187500,0
5292,힐리,KT,0.252,0.383,27.0,11.0,0.407407,0.176471,0.194,0.50,0.083333,0
5293,힐리,KT,0.256,0.385,29.0,11.0,0.379310,0.071429,0.206,0.50,0.000000,1
5294,힐리,KT,0.257,0.384,33.0,13.0,0.393939,0.142857,0.235,0.50,0.142857,1


In [16]:
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.preprocessing import minmax_scale
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
mlp_train, mlp_test = train_test_split(mlp_data, test_size=0.2, random_state=30)

In [19]:
X_train = mlp_train.loc[:, ['avg', 'slg', 'vs_ab', 'vs_hit', 'vs_avg', 'recent_5','recent_10','success_recent_10','recent_days_avg']]
y_train = mlp_train.loc[:, ['result']]

X_test = mlp_test.loc[:, ['avg', 'slg', 'vs_ab', 'vs_hit', 'vs_avg', 'recent_5','recent_10','success_recent_10','recent_days_avg']]
y_test = mlp_test.loc[:, ['result']]

In [24]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [25]:
number_of_train_samples = X_train.shape[0]
width = X_train.shape[1]
X_train = X_train.reshape(number_of_train_samples, width)

In [28]:
number_of_test_samples = X_test.shape[0]
X_test = X_test.reshape(number_of_test_samples, width)

In [29]:
X_train = minmax_scale(X_train, feature_range=(0,1), axis = 0)
X_test = minmax_scale(X_test, feature_range=(0,1), axis = 0)

In [30]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [44]:
model = Sequential()

model.add(Dense(512, input_dim= width, kernel_initializer = 'glorot_uniform', activation = 'relu'))
# model.add(Dropout(0.3))
model.add(Dense(300, kernel_initializer = 'glorot_uniform', activation = 'relu'))
model.add(Dense(256, kernel_initializer = 'glorot_uniform', activation = 'relu'))

model.add(Dense(128, kernel_initializer = 'glorot_uniform', activation = 'relu'))

model.add(Dense(64, kernel_initializer = 'glorot_uniform', activation = 'relu'))

model.add(Dense(32, kernel_initializer = 'glorot_uniform', activation = 'relu'))
model.add(Dense(16, kernel_initializer = 'glorot_uniform', activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#print('evaluate model')
#evaluation = model.evaluate(X_test, y_test, batch_size = batch_size)
#print('Accuracy: ' + str(evaluation[1]))


In [45]:
hist = model.fit(X_train, y_train, epochs= 100, batch_size = 32)

Epoch 1/100
133/133 [==============================] - 2s 5ms/step - loss: 0.6833 - accuracy: 0.6131
Epoch 2/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6728 - accuracy: 0.5986
Epoch 3/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6678 - accuracy: 0.6095
Epoch 4/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6733 - accuracy: 0.5958
Epoch 5/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6634 - accuracy: 0.6171
Epoch 6/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6739 - accuracy: 0.5919
Epoch 7/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6714 - accuracy: 0.5988
Epoch 8/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6664 - accuracy: 0.6086
Epoch 9/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6664 - accuracy: 0.6069
Epoch 10/100
133/133 [==============================] - 1s 5ms/step - loss: 0.6643 - accura

In [46]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size = 32)

34/34 [==============================] - 0s 2ms/step - loss: 0.5815 - accuracy: 0.6953
